# Preprocessing Data from Wheeloratings.com

Fortunately within wheelo ratings' brownlow prediction csv they included coaches votes and supercoach score, so this is the source for these data points

unfortunately the download button is not an actual link, so I could not automate the download.

https://www.wheeloratings.com/afl_brownlow.html

In [1]:
import pandas as pd

In [41]:
# wheeloratings have a csv with uptodate coaches votes announced on awards night
df = pd.read_csv('../../data/landing/wheelo-brownlow-predictions.csv')

In [42]:
df.head()

,Round,Match,Player,Team,Rank,Votes,Votes3,Votes2,Votes1,CoachesVotes,RatingPoints,Supercoach,Disposals,ContestedPossessions,Clearances,MetresGained,Goals,ScoreInvolvements
0,1,Rich v Carl,Daniel Rioli,Rich,1,2.007,40,21,39,10,17.6,126,27,6,0,681,0,5
1,1,Rich v Carl,Tim Taranto,Rich,2,1.801,30,22,47,3,7.2,103,32,12,5,542,0,4
2,1,Rich v Carl,George Hewett,Carl,3,0.810,11,21,6,3,12.4,89,28,14,8,122,0,3
3,1,Rich v Carl,Toby Nankervis,Rich,4,0.490,6,14,3,4,11.1,112,14,11,5,170,0,3
4,1,Rich v Carl,Dion Prestia,Rich,5,0.171,2,5,1,4,7.9,72,23,13,5,316,0,3


In [43]:
df.columns

Index(['Round', 'Match', 'Player', 'Team', 'Rank', 'Votes', 'Votes3', 'Votes2',
       'Votes1', 'CoachesVotes', 'RatingPoints', 'Supercoach', 'Disposals',
       'ContestedPossessions', 'Clearances', 'MetresGained', 'Goals',
       'ScoreInvolvements'],
      dtype='object')

In [44]:
df = df[['Round', 'Player', 'CoachesVotes', 'Supercoach']]

In [45]:
df.head()

,Round,Player,CoachesVotes,Supercoach
0,1,Daniel Rioli,10,126
1,1,Tim Taranto,3,103
2,1,George Hewett,3,89
3,1,Toby Nankervis,4,112
4,1,Dion Prestia,4,72


In [46]:
for col in df.columns:
    print(f'"{col}": "{col.lower()}",')

"Round": "round",
"Player": "player",
"CoachesVotes": "coachesvotes",
"Supercoach": "supercoach",


In [47]:
df = df.rename(columns={
    "Round": "round_number",
    "Player": "player",
    "CoachesVotes": "coaches_votes",
    "Supercoach": "SC"
})

In [48]:
df

,round_number,player,coaches_votes,SC
0,1,Daniel Rioli,10,126
1,1,Tim Taranto,3,103
2,1,George Hewett,3,89
3,1,Toby Nankervis,4,112
4,1,Dion Prestia,4,72
...,...,...,...,...
9517,24,Ryan Maric,0,49
9518,24,Jack Petruccelle,0,36
9519,24,Jayden Hunt,0,55
9520,24,Jack Williams,0,33


In [49]:
# all values filled
df.isna().sum().sort_values(ascending=False)

round_number     0
player           0
coaches_votes    0
SC               0
dtype: int64

In [50]:
# change name so can be merged in future notebook
df.loc[df['player'].str.contains("Willie"), 'player'] = "Junior Rioli"

In [51]:
players = pd.read_parquet('../../data/raw/player_information_23.parquet')

In [52]:
players['player'] = players['player_first_name'] + ' ' + players['player_last_name']

In [53]:
players.query('player.str.contains("Rioli")')

,playerid,player_first_name,player_last_name,team_name,player
14,CD_I1009551,Maurice,Rioli,Richmond,Maurice Rioli
18,CD_I1000981,Daniel,Rioli,Richmond,Daniel Rioli
153,CD_I296225,Junior,Rioli,Port Adelaide,Junior Rioli


In [54]:
df = pd.merge(df, players,
         on='player',
         how='left')

In [55]:
# merge successfully
df.loc[df.isna().any(axis=1)]

,round_number,player,coaches_votes,SC,playerid,player_first_name,player_last_name,team_name


In [60]:
df = df[['round_number', 'player', 'playerid', 'team_name', 'coaches_votes', 'SC']]

In [61]:
df.to_parquet('../../data/raw/2023_coaches_SC.parquet')